In [1]:
from ovg_experiments.simulated_data import (
    generate_simulated_data, DataGenSettings, 
    ExperimentType, SimulatedData
)
from os.path import join
from pathlib import Path
from typing import Dict

from ovg.predictors import (
    MarginalPredictor, ProposedPredictor, 
    OptimalPredictor, ImputedPredictor,
    Predictor, PredictorType
)
from ovg.evaluation import compute_zero_shot_loss, visualize_zero_shot
import numpy as np
import torch
import random
import logging
import sys

In [2]:
# Configuration

logging.basicConfig(
    level=logging.INFO,
    handlers=[logging.StreamHandler(sys.stdout)],
)
logger = logging.getLogger("zero shot learning")

experiment_type = ExperimentType.polynomial

results_dir = (                                                                                                                                                                                                               
        Path.cwd() / experiment_type.name                                                                                                                                            
)                                                                      
results_dir.mkdir(parents=True, exist_ok=True)

datagen_settings = DataGenSettings(
        num_samples=100000,                                                                                                                                                                                                        
        split_fraction=0.9,                                                                                                                                                                                                       
        noise_var=0.1,                                                                                                                                                                                                            
        noise_skew=0.0,                                                                                                                                                                                                           
        noise_mean=0.0
) 

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [3]:
# Generate Dataset

dataset: SimulatedData = generate_simulated_data(experiment_type, datagen_settings)

In [4]:
# fitting predictors

predictors: Dict[PredictorType,Predictor] = {
        PredictorType.oracle: OptimalPredictor(),
        PredictorType.marginal: MarginalPredictor(),
        PredictorType.proposed: ProposedPredictor(),
        PredictorType.imputation: ImputedPredictor()
}

for predictor_type, predictor in predictors.items():
    logger.info(f"fitting {predictor_type.name}")
    predictor.fit(dataset.data_source, dataset.data_target)

INFO:zero shot learning:fitting oracle
INFO:ovg-estimator-cond-mean:Training neural network to estimate a conditional mean
INFO:ovg-training:Epoch: 2	Loss: 0.0184
INFO:ovg-training:Epoch: 4	Loss: 0.0183
INFO:ovg-training:Epoch: 6	Loss: 0.0186
INFO:ovg-training:Epoch: 8	Loss: 0.0179
INFO:ovg-training:Epoch: 10	Loss: 0.0184
INFO:zero shot learning:fitting marginal
INFO:ovg-estimator-cond-mean:Training neural network to estimate a conditional mean
INFO:ovg-training:Epoch: 2	Loss: 1.4272
INFO:ovg-training:Epoch: 4	Loss: 1.4252
INFO:ovg-training:Epoch: 6	Loss: 1.4239
INFO:ovg-training:Epoch: 8	Loss: 1.4235
INFO:ovg-training:Epoch: 10	Loss: 1.4230
INFO:zero shot learning:fitting proposed
INFO:ovg-estimator-cond-mean:Training neural network to estimate a conditional mean
INFO:ovg-training:Epoch: 2	Loss: 1.4282
INFO:ovg-training:Epoch: 4	Loss: 1.4252
INFO:ovg-training:Epoch: 6	Loss: 1.4239
INFO:ovg-training:Epoch: 8	Loss: 1.4228
INFO:ovg-training:Epoch: 10	Loss: 1.4227
INFO:ovg-estimator-cond-

In [5]:
# compute zero-shot loss
losses: Dict[PredictorType, float] = compute_zero_shot_loss(
    predictors[PredictorType.oracle], 
    predictors, 
    dataset.data_target, 
    num_samples=1000
)
for predictor_type, loss in losses.items():
    print(f"{predictor_type.name}\t\t{loss:.3f}")

INFO:ovg-zero-shot-loss:loss PredictorType.oracle:	0.000
INFO:ovg-zero-shot-loss:loss PredictorType.marginal:	1.487
INFO:ovg-zero-shot-loss:loss PredictorType.proposed:	0.011
INFO:ovg-zero-shot-loss:loss PredictorType.imputation:	1.483
oracle		0.000
marginal		1.487
proposed		0.011
imputation		1.483


In [6]:
# plotting

visualize_zero_shot(
    predictors, dataset.data_target, save_dir=results_dir, num_samples=1000,
    display=True, figsize = (48,48), dpi=200, fontsize=30
)

print(f"plots saved in {results_dir}")

INFO:ovg-visualize_zero_shot:Visualizing oracle predictor
INFO:ovg-visualize_zero_shot:Visualizing marginal predictor
INFO:ovg-visualize_zero_shot:Visualizing proposed predictor
INFO:ovg-visualize_zero_shot:Visualizing imputation predictor


plots saved in /home/vberenz/Workspaces/ovg/Out-of-Variable-Generalization/experiments/polynomial
